# groundup_005_forward_backward_passes

In [ ]:
#| default_exp delete0004

In [ ]:
#| export
a = "to delete"

## imports

In [ ]:
from fastdebug.utils import *

In [ ]:
from fastdebug.groundup import *

In [ ]:
# %whos function

In [ ]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor

In [ ]:
# fastnbs("set_printoptions")

In [ ]:
mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
np.set_printoptions(precision=2, linewidth=140)

## get data: get_exp_data, map, tensor

In [ ]:
# fastnbs("idx check")
# check(get_exp_data)

In [ ]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'

In [ ]:
Path

pathlib.Path

In [ ]:
# get_exp_data??

In [ ]:
x_train, y_train, x_valid, y_valid = map(tensor, get_exp_data())

In [ ]:
x_train.shape
type(x_train)

torch.Size([50000, 784])

torch.Tensor

## Foundations version

In [ ]:
# fastlistnbs("groundup")


### w1, b1, w2, b2

In [ ]:
# n,m = x_train.shape
# c = y_train.max()+1
# n,m,c

In [ ]:
type(x_train)

torch.Tensor

In [ ]:
input_r,input_c = x_train.shape
label_num = y_train.max()+1
input_r
input_c
label_num

50000

784

tensor(10)

In [ ]:
# num hidden activations
nh = 50

In [ ]:
# w1 = torch.randn(m,nh)
# b1 = torch.zeros(nh)
# w2 = torch.randn(nh,1)
# b2 = torch.zeros(1)

In [ ]:
w1 = torch.randn(input_c,nh) # weights or coefficients for input
b1 = torch.zeros(nh) # biases
w2 = torch.randn(nh,1) # weights for hidden activations 
b2 = torch.zeros(1)
w1.shape
b1.shape
w2.shape
b2.shape

torch.Size([784, 50])

torch.Size([50])

torch.Size([50, 1])

torch.Size([1])

### lin(x,w,b)

In [ ]:
#| export groundup
def lin(x, w, b): 
    "build a single layer linear model. use torch.matmul (faster version of einsum) to create a linear model"
    return x@w + b

In [ ]:
hidden_layer_activations = lin(x_train, w1, b1)
hidden_layer_activations.shape

torch.Size([50000, 50])

In [ ]:
# t = lin(x_valid, w1, b1)
# t.shape

### relu(x)

In [ ]:
#| export groundup
def relu(x): 
    "basic relu with max in torch"
    return x.clamp_min(0.)

In [ ]:
t = relu(lin(x_valid, w1, b1)) # add relu unto the linear model
t

tensor([[ 7.88,  0.00,  0.00,  ...,  1.39,  0.00, 13.54],
        [ 0.00,  0.00,  0.00,  ...,  5.91,  6.00,  0.00],
        [ 5.73,  0.00,  0.00,  ...,  0.00,  0.00,  0.00],
        ...,
        [ 0.94,  1.85,  0.00,  ...,  0.68,  2.46, 13.75],
        [ 2.64,  0.00,  0.00,  ...,  0.00,  0.00,  2.18],
        [ 4.90,  0.00,  6.56,  ...,  1.46,  0.00,  4.66]])

In [ ]:
t.shape

torch.Size([10000, 50])

In [ ]:
test_eq((t >= 0).count_nonzero(), 500000)
test_eq((t < 0).count_nonzero(), 0)

### model(x_valid)

In [ ]:
def model(xb):
    "build a model of 2 layers (1 hidden layer) using lin and relu"
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    res = lin(l2, w2, b2)
    return res

In [ ]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### MSE (mean of the squared error)

We need to get rid of that trailing (,1), in order to use `mse`.

In [ ]:
res[:,0].shape

torch.Size([10000])

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [ ]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [ ]:
y_train, y_valid

(tensor([5, 0, 4,  ..., 8, 4, 8]), tensor([3, 8, 6,  ..., 5, 6, 8]))

In [ ]:
y_train,y_valid = y_train.float(),y_valid.float()

In [ ]:
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [ ]:
mse(preds, y_train)

tensor(811.88)

### Gradients and backward pass

### d(MSE)/d(error)
derivate of MSE with respect to error

In [ ]:
from sympy import symbols,diff

In [ ]:
import sympy

In [ ]:
err,tot = symbols('err,tot') # err as error, tot as total
expr = err**2/tot # mean of squared errror
l = len(x_train)
expr
expr1 = expr.subs({tot:l})
diff(expr, err)
diff(expr1, err)

err**2/tot

2*err/tot

err/25000

### d(error)/d(out)

In [ ]:
out, targ = symbols("out, targ")
diff(out - targ, out)

1

### question: gradients of input, w and b

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [ ]:
w1.shape
w1.t().shape

torch.Size([784, 50])

torch.Size([50, 784])

In [ ]:
from sympy import MatMul, MatrixSymbol
A = MatrixSymbol('A', 5, 4)
B = MatrixSymbol('B', 4, 3)
C = MatrixSymbol('C', 3, 6)
MatMul(A, B, C).shape

(5, 6)

In [ ]:
diff(MatMul(A, B), B)
diff(MatMul(A, B), A)

PermuteDims(ArrayTensorProduct(A.T, I), (3)(1 2))

PermuteDims(ArrayTensorProduct(I, B), (3)(1 2))

### question: manual derivaties for forward_backward

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    diff = out[:,0]-targ
#     loss = res.pow(2).mean()
    loss = diff.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g # derivate of relu(l1) with respect to l1
    lin_grad(inp, l1, w1, b1)

In [ ]:
# forward_and_backward(x_train, y_train)

In [ ]:
# # Save for testing against later
# w1g = w1.g.clone()
# w2g = w2.g.clone()
# b1g = b1.g.clone()
# b2g = b2.g.clone()
# ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [ ]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [ ]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [ ]:
loss = forward(xt2, y_train)
loss.backward()

In [ ]:
from fastcore.test import test_close

In [ ]:
# test_close(w22.grad, w2g, eps=0.01)
# test_close(b22.grad, b2g, eps=0.01)
# test_close(w12.grad, w1g, eps=0.01)
# test_close(b12.grad, b1g, eps=0.01)
# test_close(xt2.grad, ig , eps=0.01)

## Refactor model

### Layers as classes

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
Relu()(x_train)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 438 ms, sys: 63.6 ms, total: 502 ms
Wall time: 66.7 ms


In [ ]:
%time model.backward()

CPU times: user 857 ms, sys: 240 ms, total: 1.1 s
Wall time: 165 ms


In [ ]:
# test_close(w2g, w2.g, eps=0.01)
# test_close(b2g, b2.g, eps=0.01)
# test_close(w1g, w1.g, eps=0.01)
# test_close(b1g, b1.g, eps=0.01)
# test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def bwd(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 423 ms, sys: 57.2 ms, total: 480 ms
Wall time: 66.1 ms


In [ ]:
%time model.backward()

CPU times: user 652 ms, sys: 301 ms, total: 953 ms
Wall time: 138 ms


In [ ]:
# test_close(w2g, w2.g, eps=0.01)
# test_close(b2g, b2.g, eps=0.01)
# test_close(w1g, w1.g, eps=0.01)
# test_close(b1g, b1.g, eps=0.01)
# test_close(ig, x_train.g, eps=0.01)

NameError: name 'w2g' is not defined

### Autograd

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [ ]:
model = Model(input_c, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [ ]:
l0 = model.layers[0]
l0.b.grad

tensor([    21.07,      1.18,     -1.03,     24.95,     57.64,     11.51,      0.25,      9.34,    -25.36,      4.43,    -25.75,    -41.82,
           -30.41,    -12.28,     56.52,     -9.08,     34.23,      3.00,     63.71,     14.90,      1.81,      2.72,      7.63,     25.53,
            17.54,    -16.23,    -13.44,     14.18,     17.10,      0.67,     10.15,      5.60,     -2.15,     -7.69,      8.47,    -53.24,
            15.85,     -1.76,    -17.73,      2.44,    -11.74,     -3.64,     -0.04,    -10.73,     -0.77,      8.55,      0.73,    -14.79,
            -6.69,     -2.89])